In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        flashLoans(where: {timestamp_lte: """ + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            reserve {
                symbol
            }
            target
            amount
            totalFee
            initiator {
                id
            }
            timestamp
        }
    }
    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})
    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["flashLoans"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue
    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["initiator"] = transaction["initiator"]["id"]
        transaction["reserve"] = transaction["reserve"]["symbol"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1

with open('flashLoan.csv', 'w', newline='') as output_file:
    DICT_WRITER = csv.DictWriter(output_file, keys)
    DICT_WRITER.writeheader()
    DICT_WRITER.writerows(transactions)
        

1000 transactions found at timestamp 1654016400
999 transactions found at timestamp 1652331328
999 transactions found at timestamp 1650796561
999 transactions found at timestamp 1646533273
998 transactions found at timestamp 1643887436
999 transactions found at timestamp 1642247847
999 transactions found at timestamp 1639156030
999 transactions found at timestamp 1636736991
999 transactions found at timestamp 1633718454
998 transactions found at timestamp 1632125152
999 transactions found at timestamp 1629877927
999 transactions found at timestamp 1628449093
999 transactions found at timestamp 1627267724
999 transactions found at timestamp 1626296187
999 transactions found at timestamp 1625302586
999 transactions found at timestamp 1624576010
999 transactions found at timestamp 1624272162
999 transactions found at timestamp 1623676332
999 transactions found at timestamp 1623129409
998 transactions found at timestamp 1622339506
999 transactions found at timestamp 1621826967
996 transact

In [2]:
!wc -l flashLoan.csv
# 29449

29869 flashLoan.csv


In [3]:
from google.colab import files
files.download('flashLoan.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>